In [1]:
import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 



## Working code

In [29]:
#REMOVING THE TIME FUNCTION COMPLETELY FROM THE BENDER'S DECOMPOSITION:


#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 13:00'
end_date='2018-05-28 13:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Since we are not looking at periods we give a single value to Solar_p: can create scenarios when algorithm works
Solar_p = 17
Market = 60

#--Constants--
Constants= {
    'Load_penalty':200, 
    'Hydro_cap':50, 
    'Load':150,
    'Market':60,
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i] for i in model.plants) - sum(model.Mi[i]*model.p[i] for i in model.plants) + model.alpha #minimize(power produced - power sold to market)
    
    #return sum(model.yi[i] * model.p[i] for i in model.plants) + sum(model.ki[s] * model.phi[s] for s in model.solar) + sum(model.Li[n] * model.L_p[n] for n in model.penalty) + model.alpha

def p_bounds(model,i):
    return (model.Pmin[i], model.Pmax[i])   


def Hydro_capacity(model, i):
    return model.p[i] <= model.H_stage1[i]

def CreateCuts(model,c):
    return(model.alpha >= model.Phi[c] - sum(model.Lambda[c,i]*(model.p[i]-model.x_hat[c,i]) for i in model.plants))  #only takes into account the hydro power production scheduling from H_cap

#model.Phi[c] IS the Obj_2nd function result!. Need to include all the complicating variables with their sensitivities as well                                                                                                                      

# ----- Second stage ojbective and constraints --------------#
def Obj_2nd(model):
    return sum((model.yi[i]*model.p[i] + 0*model.phi + model.Li * model.L_p) for i in model.plants)
#p is a parameter in the second objective

def Solar_rule(model):
    return model.phi == Solar_p

def Hydro_capacity2(model, i):      #ensures that model.p does not use more capacity than what it has available
    return model.p[i] <= model.H_stage2[i]

def load_rule(model):
    return model.p['Hydro1'] + model.p['Hydro2'] + model.phi + model.L_p == model.L 
  
#def hydro_prod(model,i):
#    return model.p[i] == model.X_hat[i]


def calculate_initial_H_stage2(Constants, X_hat):
    initial_H_stage2 = {}
    for plant in X_hat:
        remaining_capacity = Constants['Hydro_cap'] - (X_hat[plant])
        initial_H_stage2[plant] = remaining_capacity
    return initial_H_stage2

# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants   =  Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')

    #Parameters
    model.Ci       =  pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.yi       =  pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin     =  pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax     =  pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage1 =  pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Initial capacity
    model.Mi       =  pyo.Param(model.plants, initialize=Constants['Market'])                           #market sell price
    model.L        =  pyo.Param(initialize=Constants['Load'])

    # Variables
    model.p        =  pyo.Var(model.plants, bounds=p_bounds)  # Production from hydro plants

    # Constraints
    model.hydro_cons   =  pyo.Constraint(model.plants, rule=Hydro_capacity)

    # Cuts information
    model.Cut      =  pyo.Set(initialize = Cuts["Set"])
    model.Phi      =  pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda   =  pyo.Param(model.Cut, model.plants, initialize = Cuts["lambda"])
    model.x_hat    =  pyo.Param(model.Cut, model.plants, initialize = Cuts["x_hat"])
    
    model.alpha    =  pyo.Var(bounds = (-100000000,100000000))
    
    model.CreateCuts   =  pyo.Constraint(model.Cut, rule = CreateCuts)

    # Define objective function
    model.obj      =  pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model



def ModelSetUp_2nd(data, Constants, X_hat, rem_cap):
    # Instance
    model          =  pyo.ConcreteModel()

    # Define sets
    model.plants   =  Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    #model.solar    =  Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    #model.penalty  =  Set(initialize=['Load_penalty']) 

    # Define parameters
    model.X_hat    =  pyo.Param(model.plants, initialize=X_hat)
    model.Li       =  pyo.Param(initialize=Constants['Load_penalty']) 
    model.yi       =  pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin     =  pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax     =  pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage2 =  pyo.Param(model.plants, initialize=rem_cap)
    model.L        =  pyo.Param(initialize=Constants['Load'])
    
    # Define variables
    model.p        =  pyo.Var(model.plants, bounds=p_bounds)
    model.phi      =  pyo.Var(within=NonNegativeReals)
    model.L_p      =  pyo.Var(within=NonNegativeReals)

    # Constraints
    model.load_cons    =  pyo.Constraint(rule=load_rule)
    model.solar_cons   =  pyo.Constraint(rule=Solar_rule)
    model.hydro_cons   =  pyo.Constraint(model.plants, rule=Hydro_capacity2)                                           #constraining power production from hydrom from the remaining capacity

    # Define objective function
    model.obj     =  pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        Cuts["lambda"][cut, i] = model.dual.get(model.hydro_cons[i])  # the lambda should be the economic change when we increase the 1st decision by 1. Should be something else
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}


#For now:
for i in range(3):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {
            "Hydro1": m_1st.p["Hydro1"].value,
            "Hydro2": m_1st.p["Hydro2"].value,
        }
    
    display("How much capacity was used by each plant in stage 1 after solving loop: "  +  str( i ) + "  " + str(X_hat))
    DisplayResults(m_1st)
    
    #Updating remaining capacity between stages
    initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)         #calculate changing capacity between stages
    display("How much capacity is left for each plant in stage 2 after solving loop: " +  str( i ) + "  " + str(initial_H_stage2))

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)
    DisplayResults(m_2nd)
    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    print()
    print()

    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    

    tol = (pyo.value(m_1st.alpha.value)-pyo.value(m_2nd.obj))
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    #input()


"How much capacity was used by each plant in stage 1 after solving loop: 0  {'Hydro1': 40.0, 'Hydro2': 50.0}"

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  40.0 :    40 : False : False :  Reals
        Hydro2 :     0 :  50.0 :   100 : False : False :  Reals
    alpha : Size=1, Index=None
        Key  : Lower      : Value        : Upper     : Fixed : Stale : Domain
        None : -100000000 : -100000000.0 : 100000000 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -100002650.0

  Constraints:
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 40.0 :  50.0
        Hydro2 :  None : 50.0 :  50.0
    CreateCuts : Size=0
        Key : Lower : Body : Upper
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                : Value
    hydro_cons[Hydro1] :   0.0
    hydro_cons[Hydro2] : -25.0
None None


"How much capacity is left for each plant in stage 2 after solving loop: 0  {'Hydro1': 10.0, 'Hydro2': 0.0}"

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  10.0 :    40 : False : False :  Reals
        Hydro2 :     0 :   0.0 :   100 : False : False :  Reals
    phi : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  17.0 :  None : False : False : NonNegativeReals
    L_p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 123.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 49450.0

  Constraints:
    load_cons : Size=1
        Key  : Lower : Body  : Upper
        None : 150.0 : 150.0 : 150.0
    solar_cons : Size=1
        Key  : Lower : Body : Upper
        None :  17.0 : 17.0 :  17.0
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 10.0 :  10

"How much capacity was used by each plant in stage 1 after solving loop: 1  {'Hydro1': 0.0, 'Hydro2': 0.0}"

...

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  40.0 :    40 : False : False :  Reals
        Hydro2 :     0 :  50.0 :   100 : False : False :  Reals
    phi : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  17.0 :  None : False : False : NonNegativeReals
    L_p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  43.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 19950.0

  Constraints:
    load_cons : Size=1
        Key  : Lower : Body  : Upper
        None : 150.0 : 150.0 : 150.0
    solar_cons : Size=1
        Key  : Lower : Body : Upper
        None :  17.0 : 17.0 :  17.0
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 40.0 :  50

"How much capacity was used by each plant in stage 1 after solving loop: 2  {'Hydro1': 10.0, 'Hydro2': 0.0}"

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  10.0 :    40 : False : False :  Reals
        Hydro2 :     0 :   0.0 :   100 : False : False :  Reals
    alpha : Size=1, Index=None
        Key  : Lower      : Value   : Upper     : Fixed : Stale : Domain
        None : -100000000 : 19950.0 : 100000000 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 19600.0

  Constraints:
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 10.0 :  50.0
        Hydro2 :  None :  0.0 :  50.0
    CreateCuts : Size=2
        Key : Lower : Body : Upper
          0 :  None :  0.0 :   0.0
          1 :  None :  0.0 :   0.0
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                : Value
         CreateCuts[0] : -0.09333333333333334
         CreateCuts[1] :  -0.906666666666

"How much capacity is left for each plant in stage 2 after solving loop: 2  {'Hydro1': 40.0, 'Hydro2': 50.0}"

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  40.0 :    40 : False : False :  Reals
        Hydro2 :     0 :  50.0 :   100 : False : False :  Reals
    phi : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  17.0 :  None : False : False : NonNegativeReals
    L_p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  43.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 19950.0

  Constraints:
    load_cons : Size=1
        Key  : Lower : Body  : Upper
        None : 150.0 : 150.0 : 150.0
    solar_cons : Size=1
        Key  : Lower : Body : Upper
        None :  17.0 : 17.0 :  17.0
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 40.0 :  40

## Testing code

In [8]:
#REMOVING THE TIME FUNCTION COMPLETELY FROM THE BENDER'S DECOMPOSITION:


#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 13:00'
end_date='2018-05-28 13:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Since we are not looking at periods we give a single value to Solar_p: can create scenarios when algorithm works
Solar_p = 10

#Unique scenarios for stochastic variable
S_high=Solar_p*1.5
S_avg=Solar_p*1
S_low=Solar_p*0.5

#Load 
#L= {1:50} #3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

Market = 30

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':50, 
    'Load':150,
    'probs':{'S_high':1/3, 'S_avg':1/3, 'S_low':1/3},    
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i] for i in model.plants) - sum(model.m[i]*model.p[i] for i in model.plants) #minimize(power produced - power sold to market)
    
def p_bounds(model,i):
    return (model.Pmin[i], model.Pmax[i])   


def Hydro_capacity(model, i):
    return model.p[i] <= model.H_stage1[i]

def CreateCuts(model,c):
    return(model.alpha >= model.Phi[c] + sum(model.Lambda[c,i]*(model.p[i]-model.x_hat[c,i]) for i in model.plants))


# ----- Second stage ojbective and constraints --------------# -- IMPLEMENT scenarios to have Stochasticity in the second stage decision.
def Obj_2nd(model):
    return sum(model.probs[l] * (model.yi[i]*model.p[i] + model.phi[l] + model.Li[n] * model.L_p[n]) for i in model.plants for l in model.scenarios for n in model.penalty)

#model.ki[s] *

#def Solar_rule(model,l):
#    return model.phi['Solar',l] == Solar_p

def Solar_high(model):
    if model.probs['S_high']>0:
        return model.phi['S_high']==S_high
    else:
        return model.phi['S_high']==0
    
#Solar production in stage 2 must be equal to scenario for average production
def Solar_avg(model):
    if model.probs['S_avg']>0:
        return  model.phi['S_avg'] == S_avg
    else:
        return  model.phi['S_avg'] == 0
    
#Solar production in stage 2 must be equal to scenario for low production
def Solar_low(model):
    if model.probs['S_low']>0:
        return  model.phi['S_low'] == S_low
    else:
        return  model.phi['S_low'] == 0
    

def Hydro_capacity2(model, i):      #ensures that model.p does not use more capacity than what it has available
    return model.p[i] <= model.H_stage2[i]

def load_rule(model,l):
    return model.p['Hydro1'] + model.p['Hydro2'] + model.phi['Solar',l] + model.L_p['Load_penalty'] == model.L 

def calculate_initial_H_stage2(Constants, X_hat):
    initial_H_stage2 = {}
    for plant in X_hat:
        remaining_capacity = Constants['Hydro_cap'] - (X_hat[plant])
        initial_H_stage2[plant] = remaining_capacity
    return initial_H_stage2

# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')

    #Parameters
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.H_stage1 = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])     #Initial capacity
    model.m = pyo.Param(model.plants, initialize=Market)   #market sell price

    model.L = pyo.Param(initialize=Constants['Load'])


    # Variables
    model.p   = pyo.Var(model.plants, bounds=p_bounds)  # Production from hydro plants

    # Constraints
    model.hydro_cons=pyo.Constraint(model.plants, rule=Hydro_capacity)

    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.plants, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.plants, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)


    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model


def ModelSetUp_2nd(data, Constants, X_hat, rem_cap):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    #model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar') ->used for solar cost
    model.penalty= Set(initialize=['Load_penalty']) 
    model.scenarios = Set(initialize=Constants['probs'].keys())

    # Define parameters
    model.X_hat = pyo.Param(model.plants, initialize=X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    #model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])  #solar is free and will always be utilizied
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    #model.Phi_min=pyo.Param(model.scenarios, initialize=data['solar']['P_min'])
    #model.Phi_max=pyo.Param(model.scenarios, initialize=data['solar']['P_max'])
    model.H_stage2 = pyo.Param(model.plants, initialize=rem_cap)
    model.probs=pyo.Param(model.scenarios, initialize=Constants['probs'])       #Scenarios indexed by "l"
    model.L = pyo.Param(initialize=Constants['Load'])
    
    # Define variables
    model.p   = pyo.Var(model.plants, bounds=p_bounds)
    model.phi = pyo.Var(model.scenarios, within=NonNegativeReals)
    model.L_p = pyo.Var(model.penalty, within=NonNegativeReals)

    #Constraints
    model.high_sun= pyo.Constraint(rule=Solar_high)
    model.avg_sun= pyo.Constraint(rule=Solar_avg)
    model.low_sun= pyo.Constraint(rule=Solar_low)

    model.load_cons  = pyo.Constraint(model.scenarios, rule=load_rule)
    model.hydro_cons = pyo.Constraint(model.plants, rule=Hydro_capacity2)  #constraining power production from hydrom from the remaining capacity
    
    #Stochastic constraints

    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        Cuts["lambda"][cut, i] = model.dual.get(model.hydro_cons[i])  # the lambda should be the economic change when we increase the 1st decision by 1, => dual value of hydro prod
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}

#X_hat = {
#    "Hydro1": 0.0,
#    "Hydro2": 0.0,
#}

#This is the while-loop in principle, but for this case is only a for-loop

#CODE FOR LATER WHEN LOGIC WORKS:
#tol = 1e-4     #tolerance for while loop
#while (pyo.value(m_1st.alpha.value - pyo.value(m_2nd.obj)) >= tol): #while loop that doesn't end until upper bound and lower bound converge (not working so something similar)



#For now:
for i in range(5):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {
            "Hydro1": m_1st.p["Hydro1"].value,
            "Hydro2": m_1st.p["Hydro2"].value,

        }
    display(X_hat)
    DisplayResults(m_1st)
    
    #Updating remaining capacity between stages
    initial_H_stage2 = calculate_initial_H_stage2(Constants, X_hat)         #calculate changing voluem
    display("How much capacity is left for each plant for stage 2 after solving loop: " +  str( i ) + "  " + str(initial_H_stage2))

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, initial_H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)
    DisplayResults(m_2nd)
    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    print()
    print()

    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    #input()


{'Hydro1': 40.0, 'Hydro2': 0.0}

Model unknown

  Variables:
    p : Size=2, Index=plants
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Hydro1 :     0 :  40.0 :    40 : False : False :  Reals
        Hydro2 :     0 :   0.0 :   100 : False : False :  Reals
    alpha : Size=1, Index=None
        Key  : Lower      : Value : Upper     : Fixed : Stale : Domain
        None : -100000000 :  None : 100000000 : False :  True :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : -200.0

  Constraints:
    hydro_cons : Size=2
        Key    : Lower : Body : Upper
        Hydro1 :  None : 40.0 :  50.0
        Hydro2 :  None :  0.0 :  50.0
    CreateCuts : Size=0
        Key : Lower : Body : Upper
dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                : Value
    hydro_cons[Hydro1] :   0.0
    hydro_cons[Hydro2] :   0.0
None None


"How much capacity is left for each plant for stage 2 after solving loop: 0  {'Hydro1': 10.0, 'Hydro2': 50.0}"

ERROR: Rule failed when generating expression for Constraint load_cons with
index S_high: KeyError: "Index '('Solar', 'S_high')' is not valid for indexed
component 'phi'"
ERROR: Constructing component 'load_cons' from data=None failed: KeyError:
"Index '('Solar', 'S_high')' is not valid for indexed component 'phi'"


KeyError: "Index '('Solar', 'S_high')' is not valid for indexed component 'phi'"

In [ ]:
probs = Constants['probs']
print(a)
for l in probs:
    print(l)

print(probs['S_high'])

{'S_high': 0.3333333333333333, 'S_avg': 0.3333333333333333, 'S_low': 0.3333333333333333}
S_high
S_avg
S_low
0.3333333333333333


## Scrapped time period code

In [ ]:
#--Data handling--
#Read of parameters for portfolio
def InputData(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['hydro'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Solar')
    data['solar']=inputdata[['Ci', 'yi', 'P_min', 'P_max']].drop('Hydro1').drop('Hydro2')
    return data

data = InputData('data/Parameters.xlsx')


#Solar production based on forecast 
def read_solar_data(irrad_file, spec_file, start_date, end_date):
    input_data_PV = read_excel_data(spec_file)
    input_data_Irr = read_irr_data(irrad_file)
    PV_power = pv_power_estimated(input_data_PV,input_data_Irr)
    Solar_1=convert_to_dict(PV_power, start_date, end_date, 'H')
    Solar_p=scale_dict(Solar_1, 1)
    return Solar_p

#Start and end dates of the optimization
start_date='2018-05-28 13:00'
end_date='2018-05-28 14:00'     #Two time steps only

#Original forecast for solar power production
Solar_p=read_solar_data('data/Data_solar_irr_NOR.csv', 'data/PV_spec.xlsx', start_date, end_date)

#Load 
L= {1:50, 2:50} #3:20, 4:30, 5:50, 6:80, 7:50, 8:90, 9:110, 10:150, 11:120, 12:80, 13:70, 14:80, 15:90, 16:160, 17:170, 18:150, 19:120, 20:100, 21:70, 22:60, 23:50, 24:40} 

#--Constants--
Constants= {
    'Load_penalty':100, 
    'Hydro_cap':40, 
}

# ----------------------Mathematical formulation ------------------------- #
# ----- First stage obejctive and constraints ------------------ #
def Obj_1st(model):
    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) + sum(model.ki[s] * model.phi[s, j] for s in model.solar for j in model.periods) + sum(model.Li[n] * model.L_p[n,j] for n in model.penalty for j in model.periods) + model.alpha
#def Obj_1st(model):
#    return sum(model.yi[i] * model.p[i, j] for i in model.plants for j in model.periods) + sum(model.ki * model.phi[s, j] for s in model.solar for j in model.periods) + sum(model.Li[n] * model.L_p[n, j] for n in model.penalty for j in model.periods) + model.alpha 
#Production bounds for hydro plants in stage 1 and stage 2
def p_bounds(model,i,j):
    return (model.Pmin[i],model.Pmax[i])   
#Load demand in first stage and second stage
def load_rule(model, j):
    return model.p['Hydro1', j] + model.p['Hydro2', j] + model.phi['Solar', j] + model.L_p['Load_penalty',j] == L[j]
#Solar production in stage 1 must be equal to original forecast
def Solar_rule(model,j):
    return  model.phi['Solar',j] == Solar_p[j]
#Sum of hydro production in stage 1 must be lower than available generation
#def Hydro_capacity(model,i):
#    return sum(model.p['Hydro1',j] + model.p['Hydro2',j] for j in model.periods)<=model.Hmax[i]

def Hydro_capacity(model, i, j):
    return model.p['Hydro1', j] + model.p['Hydro2', j] <= model.H_stage1[i]

def Remaining_Capacity_first(model, i, j):
    return model.H_stage2[i] == model.H_stage1[i] - model.p['Hydro1', j] - model.p['Hydro2', j]        #LINKING CONSTRAINT, how much capacity is left for stage 2 after stage 1
    
def CreateCuts(model,Ci):
    return(model.alpha <= model.Phi[Ci] + sum(model.Lambda[Ci,i]*(model.p[i]-model.x_hat[Ci,i]) for i in model.plants))  #only takes into account the hydro power production scheduling from H_cap
                                                                                                                         
#def CreateCuts(m,c):
#    return(m.alpha <= m.Phi[c] + sum(m.Lambda[c,i]*(m.x[i]-m.x_hat[c,i]) for i in m.I)) c = crop cost.... should be similar to how we obtain power produciton cost Ci

#Divide x_hat into "hydro", "solar" and "Load_penalty" as well! NO don't think so, only hydro1 and hydro2 affect the capacity H!!


# ----- Second stage ojbective and constraints --------------#
def Obj_2nd(model):
    return sum((model.yi[i])*(model.p[i,j] + model.ki[s] * model.phi[s, j] + model.Li[n] * model.L_p[n,j]) for i in model.plants for s in model.solar for n in model.penalty for j in model.periods)

def Hydro_capacity2(model, i, j):
    return model.p['Hydro1', j] + model.p['Hydro2', j] <= model.H_stage2[i]             #new constraint which does not allow the power production from model.p to be higher than remaining cap

#def Crop_plant(m,i):
#    return m.x[i] == m.X_hat[i]            # Do we need the X_hat values now? :I I would think so
#X_hat values indicate how much hydro power, solar power and load penalties was apparent in stage 1. It should add to the constraint in stage 2 somehow, at least the used hydro power

#Alternative remaining cap calculation:
#def rem_cap(model, i)
#    remaining_cap = 0
#    return remaining_cap = X_hat['hydro'] 
def rem_cap(Constants, X_hat):
    remaining_cap = 0
    return remaining_cap == sum(Constants['Hydro_cap',j]) - sum((X_hat['Hydro1',j] + X_hat['Hydro2',j])) 



# ------------------------- MODEL SETUP -------------------------- #

# Set up model 1st stage
def ModelSetUp_1st(data, Constants,Cuts):
    model = ConcreteModel()

    # Define sets and parameters specific to the first stage
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys())   # Set of time periods (e.g., 1, 2, ..., 24)
    model.penalty= Set(initialize=['Load_penalty']) 
    #Parameters
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])

    #Maximum hydro production available for the timesteps
    #model.Hmax = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])
    model.H_stage1 = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])         #Initial capacity
    model.H_stage2 = pyo.Param(model.plants, initialize=Constants['Hydro_cap'])         #Should be updated due to remaining_capacity_first constraint, input for remaining capacity in stage 2

    # Load data
    model.L = Param(model.periods, initialize=lambda model, j: L[j])
    
    # Variables
    model.p   = pyo.Var(model.plants, model.periods, bounds=p_bounds)  # Production from hydro plants
    model.phi = pyo.Var(model.solar, model.periods, within=NonNegativeReals)  # Production from solar installation
    model.L_p = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)  # Load penalty
    #model.cap = pyo.Var(model.plants, within=NonNegativeReals, initialize=Constants['Hydro_cap'])

    
    # Constraints
    model.pbounds = pyo.Constraint(model.plants, model.periods, rule = p_bounds)
    model.load_cons = pyo.Constraint(model.periods, rule=load_rule)
    model.hydro_cons=pyo.Constraint(model.plants, model.periods, rule=Hydro_capacity)
    model.remaining_cap = pyo.Constraint(model.plants, model.periods, rule=Remaining_Capacity_first)


    """Cuts_information"""
    #Set for cuts
    model.Cut = pyo.Set(initialize = Cuts["Set"])
    #Parameter for cuts
    model.Phi = pyo.Param(model.Cut, initialize = Cuts["Phi"])
    model.Lambda = pyo.Param(model.Cut, model.periods, initialize = Cuts["lambda"])
    model.x_hat = pyo.Param(model.Cut, model.periods, initialize = Cuts["x_hat"])
    #Variable for alpha
    model.alpha = pyo.Var(bounds = (-100000000,100000000))
    
    """Constraint cut"""
    model.CreateCuts = pyo.Constraint(model.Cut, rule = CreateCuts)


    """Constraints"""
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_1st, sense=pyo.minimize)
    
    return model



def ModelSetUp_2nd(data, Constants, X_hat):
    # Instance
    model = pyo.ConcreteModel()
    # Define sets
    model.plants = Set(initialize=data['hydro'].index)  # Set of plant types (e.g., 'Hydro1', 'Hydro2')
    model.solar = Set(initialize=data['solar'].index)  # Set of solar types (e.g., 'Solar')
    model.periods = Set(initialize=L.keys()) 
    model.penalty= Set(initialize=['Load_penalty']) 

    # Define parameters
    model.X_hat = pyo.Param(model.plants, model.periods, initialize=X_hat)
    model.Li=pyo.Param(model.penalty, initialize=Constants['Load_penalty'])
    model.Ci=pyo.Param(model.plants, initialize=data['hydro']['Ci'])
    model.Si=pyo.Param(model.solar, initialize=data['solar']['Ci'])
    model.yi=pyo.Param(model.plants, initialize=data['hydro']['yi'])
    model.ki=pyo.Param(model.solar, initialize=data['solar']['yi'])
    model.Pmin=pyo.Param(model.plants, initialize=data['hydro']['P_min'])
    model.Pmax=pyo.Param(model.plants, initialize=data['hydro']['P_max'])
    model.Phi_min=pyo.Param(model.solar, initialize=data['solar']['P_min'])
    model.Phi_max=pyo.Param(model.solar, initialize=data['solar']['P_max'])
    model.H_stage2 = pyo.Param(model.plants, initialize=H_stage2)

    # Define variables
    model.p   = pyo.Var(model.plants, model.periods, within=p_bounds)
    model.phi = pyo.Var(model.solar, model.periods, within=NonNegativeReals)
    model.L_p = pyo.Var(model.penalty, model.periods, within=NonNegativeReals)

    #Constraints
    model.pbounds = pyo.Constraint(model.plants, rule=p_bounds)
    model.load_cons  = pyo.Constraint(rule=load_rule)
    model.solar_constraint = pyo.Constraint(rule=Solar_rule)
    model.remaining_capacity = pyo.Constraint(model.plants, rule=rem_cap)                                           #LINKING CONSTRAINT
    model.hydro_cons = pyo.Constraint(model.plants, rule=Hydro_capacity2)                                           #constraining power production from hydrom from the remaining capacity
    
    
    # Define objective function
    model.obj = pyo.Objective(rule=Obj_2nd, sense=pyo.minimize)
    return model


def Solve(model):
    opt = SolverFactory("gurobi")
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model, load_solutions=True)
    return results, model
def DisplayResults(model):
    return print(model.display(), model.dual.display())


# ----------------------- CUTTING ------------------------ #

# Function for creating new linear cuts for optimization problem
def Cut_manage(Cuts,model):
    """Add new cut to existing dictionary of cut information"""
    
    #Find cut iteration by checking number of existing cuts
    cut = len(Cuts["Set"])
    #Add new cut to list, since 0-index is a thing this works well
    Cuts["Set"].append(cut)
    
    #Find 2nd stage cost result
    Cuts["Phi"][cut] = pyo.value(model.obj)
    #Find lambda x_hat for each type of power produced
    for i in model.plants:
        try:
            Cuts["lambda"][cut, i] = model.dual.get(model.H_stage2[i], 0)  # Use a default value if the key doesn't exist
        except KeyError:
            print(f"Key not found for {i}: {model.H_stage2[i]}")
        Cuts["x_hat"][cut,i] = model.X_hat[i]
    return(Cuts)

Cuts = {}
Cuts["Set"] = []
Cuts["Phi"] = {}
Cuts["lambda"] = {}
Cuts["x_hat"] = {}

#This is the while-loop in principle, but for this case is only a for-loop
for i in range(10):

    #Solve 1st stage problem
    m_1st = ModelSetUp_1st(data, Constants,Cuts)
    Solve(m_1st)
 
 
    # Process 1st stage result
    X_hat = {}
    for j in m_1st.periods:
        X_hat_j = {
            "Hydro1": m_1st.p["Hydro1", j].value,
            "Hydro2": m_1st.p["Hydro2", j].value,
            #"Solar":  m_1st.phi["Solar", j].value,                  #are the solar and load penalty values necessary? They are merely slack variables in  way. naaah
            #"Load_penalty": m_1st.L_p['Load_penalty', j].value
        }
        X_hat[j] = X_hat_j
    display(X_hat)

    # Setup and solve 2nd stage problem with updated H_stage1
    m_2nd = ModelSetUp_2nd(data, Constants, X_hat, m_1st.H_stage2)      #must take into account the capacity used in stage 1 as a linking variable
    Solve(m_2nd)

    #Create new cuts for 1st stage problem
    Cuts = Cut_manage(Cuts,m_2nd)
    
    #Print results 2nd stage
    print("Objective function:",pyo.value(m_2nd.obj))
    print("Cut information acquired:")
    for component in Cuts:
        if component == "lambda" or component == "x_hat":
            for j in m_2nd.plants:
                print(component,j,Cuts[component][i,j])
        else:
            print(component,Cuts[component])
    input()
    
    #We perform a convergence check
    print("UB:",pyo.value(m_1st.alpha.value),"- LB:",pyo.value(m_2nd.obj))
    input()

def displayresults(model):
    return print(model.display()) #model.dual.display())